In [2]:
import ccxt
import requests
from binance.client import Client
from binance.exceptions import BinanceAPIException
from datetime import datetime, timedelta
import time
import hashlib
import hmac
import pandas
import json


In [3]:
def get_keys(filepath):
    with open(filepath) as config_file:
        config = json.load(config_file)
        api_key = config['api_key']
        api_secret=config['api_secret']
    return api_key,api_secret

In [4]:
def get_time_off_set(filepath):
    with open(filepath) as config_file:
        time_offset=json.load(config_file)
    return time_offset

In [5]:
def synchronize_time():
    api_key,api_secret=get_keys("binance_keys.json")
    client = Client(api_key, api_secret, testnet=True)
    try:
        server_time = client.get_server_time()
        server_timestamp = server_time['serverTime']
        local_timestamp = int(time.time() * 1000)
        time_offset = server_timestamp - local_timestamp
        with open("time.json","w") as config_file:
            json.dump(time_offset,config_file)
    except Exception as e:
        print(f"An error occurred during time synchronization: {e}")

# Update to use synchronized time offset

In [6]:
get_keys("binance_keys.json")

('dIgzgeXGUjuNi81pWAZAzHt1zkYcfoN4QM0oasQsVkJQmoqdkC7dilNLiETRheyU',
 'ZxiMdpOSzIirGkhnkEBJsNHZ92okUqBwSulFIKKeHSLbLdlPkjWN9lMx5lsJn79g')

In [7]:
synchronize_time()

In [8]:
get_time_off_set("time.json")

47

In [9]:
def information_for_options(client):
    markets = client.fetch_markets()
    server_time = client.fetch_time()
    # Filter option markets
    option_markets = [market for market in markets if market['type'] == 'option']
    # print("Option markets:", option_markets)
    for i in option_markets:
        # print(i["info"])
        filtered_list = []
        expiry_timestamp = float(i['info']['expiration_timestamp'])/1000
        # Convert expiry timestamp to datetime
        expiry_datetime = datetime.utcfromtimestamp(expiry_timestamp)

        # Calculate current datetime plus 20 weeks
        current_datetime = datetime.utcnow()
        twenty_weeks_later = current_datetime + timedelta(weeks=20)
        # Check if expiry datetime is greater than 20 weeks later
        if expiry_datetime <= twenty_weeks_later:
            filtered_list.append(i)
            # if call option and strike is not none, add to list
            second_filtered_list = []
            for item in filtered_list:
                # if item["strike"] is not None and 
                if item["optionType"] == "call" and item["strike"] is not None:

                    second_filtered_list.append(item)
                    # third_filtered_list = []
                    # # Loop through second filtered list and get contract with longest expiration
                    # for item in second_filtered_list:
                    #     expired_time = item["expiration_timestamp"]
                        
    #                     if item["expiration_timestamp"] is not None:
    #                         third_filtered_list.append(item)
            third_filtered_list = []
            expiry_timestamp_list = []
            if len(second_filtered_list) > 0:
                longest_expiry_contract = max(second_filtered_list, key=lambda x: x['expiry'])

    # print(longest_expiry_contract)
    # Contract symbol
    symbol = longest_expiry_contract["id"]
    # Timestamp of contract            
    expire_time = float(longest_expiry_contract["info"]["expiration_timestamp"])
    # convert expiration timestamp to datetime
    expiry_datetime = datetime.utcfromtimestamp(expire_time/1000)
    # Get delta
    contract_delta = client.fetch_greeks(longest_expiry_contract["symbol"])
    delta = contract_delta["info"]["greeks"]["delta"]

    # Get contract size
    # Calculate share to purchase
    contract_size = longest_expiry_contract["info"]["contract_size"]
    share_to_purchase = float(delta) * float(contract_size)


    return expiry_datetime, share_to_purchase, symbol ,delta ,contract_size

In [10]:

mode = "test"

client_id_testnet = "m_cYdGRs"
client_secret_testnet = "CFMj4XERdBpYU5f-xGiEeM6q29WWPh78KbOllO8I5nI"

client_id_realnet = "VUSrWKNX"
client_secret_realnet = "CNIEmjiKy2p-h28O4Mda1QKD8hXJ3duA5rAODdLfvwE"

if mode == "test":
    client_id = client_id_testnet
    client_secret = client_secret_testnet
    base_url = 'https://test.deribit.com/api/v2/' # test base url
else:
    client_id = client_id_realnet
    client_secret = client_secret_realnet
    base_url = 'https://www.deribit.com/api/v2/'

print(base_url)


# Function to get access token
def get_access_token(client_id, client_secret):
    url = f'{base_url}public/auth'
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.get(url, params=params)
    response_data = response.json()
    if 'result' in response_data:
        return response_data['result']['access_token']
    else:
        raise Exception('Failed to get access token: ' + response_data.get('error', {}).get('message', 'Unknown error'))

# Function to fetch account information
def fetch_account_info(access_token):
    url = f'{base_url}private/get_account_summary'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'currency': 'BTC'  # or 'ETH', etc.
    }
    response = requests.get(url, headers=headers, params=params)
    response_data = response.json()
    if 'result' in response_data:
        return response_data['result']
    else:
        raise Exception('Failed to fetch account information: ' + response_data.get('error', {}).get('message', 'Unknown error'))
    
client = ccxt.deribit({
    'apiKey': client_id,
    'secret': client_secret,
    'timeout': 50000,
})
if mode=="test":
    client.set_sandbox_mode(True)

https://test.deribit.com/api/v2/


In [11]:
access_token = get_access_token(client_id, client_secret)
account_info = fetch_account_info(access_token)
print(account_info)

{'estimated_liquidation_ratio_map': {}, 'options_theta_map': {}, 'limits': {'non_matching_engine': {'burst': 15000, 'rate': 20}, 'limits_per_currency': False, 'matching_engine': {'trading': {'total': {'burst': 20, 'rate': 5}}, 'spot': {'burst': 20, 'rate': 5}, 'maximum_quotes': {'burst': 500, 'rate': 500}, 'maximum_mass_quotes': {'burst': 10, 'rate': 10}, 'guaranteed_mass_quotes': {'burst': 2, 'rate': 2}, 'cancel_all': {'burst': 20, 'rate': 5}}}, 'balance': 100.0, 'options_theta': 0.0, 'session_upl': 0.0, 'projected_delta_total': 0.0, 'options_gamma_map': {}, 'currency': 'BTC', 'maintenance_margin': 0.0, 'futures_session_rpl': 0.0, 'projected_maintenance_margin': 0.0, 'additional_reserve': 0.0, 'options_vega': 0.0, 'delta_total': 0.0, 'change_margin_model_api_limit': {'rate': 10, 'timeframe': 86400000}, 'available_withdrawal_funds': 100.0, 'options_gamma': 0.0, 'fee_balance': 0.0, 'futures_session_upl': 0.0, 'estimated_liquidation_ratio': 0.0, 'available_funds': 100.0, 'options_pl': 0.

In [12]:

def create_option_order(client):
    try:
        # Construct the order
        symbol=information_for_options(client)[-3]
        order = client.create_market_buy_order(symbol,1)

        print(f"Order created successfully: {order}")
        return order

    except Exception as e:
        print(f"Failed to create order: {e}")
        return None

# Example usage

In [13]:
expiry_datetime, share_to_purchase, symbol ,delta ,contract_size= information_for_options(client)
print(expiry_datetime,share_to_purchase, symbol)
create_option_order(client)

C:\Users\Dan_Huynh\AppData\Local\Temp\ipykernel_4252\2681487509.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  expiry_datetime = datetime.utcfromtimestamp(expiry_timestamp)
C:\Users\Dan_Huynh\AppData\Local\Temp\ipykernel_4252\2681487509.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_datetime = datetime.utcnow()
C:\Users\Dan_Huynh\AppData\Local\Temp\ipykernel_4252\2681487509.py:45: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  expiry_date

2024-09-27 08:00:00 0.0325 ETH-27SEP24-17000-C
Order created successfully: {'info': {'is_liquidation': False, 'risk_reducing': False, 'order_type': 'limit', 'creation_timestamp': '1718797294825', 'order_state': 'open', 'contracts': '1.0', 'original_order_type': 'market', 'average_price': '0.0', 'reduce_only': False, 'post_only': False, 'last_update_timestamp': '1718797294825', 'filled_amount': '0.0', 'replaced': False, 'web': False, 'api': True, 'mmp': False, 'instrument_name': 'ETH-27SEP24-17000-C', 'order_id': 'ETH-8960119361', 'max_show': '1.0', 'time_in_force': 'good_til_cancelled', 'amount': '1.0', 'direction': 'buy', 'price': '0.0685', 'label': '', 'trades': []}, 'id': 'ETH-8960119361', 'clientOrderId': None, 'timestamp': 1718797294825, 'datetime': '2024-06-19T11:41:34.825Z', 'lastTradeTimestamp': None, 'symbol': 'ETH/USD:ETH-240927-17000-C', 'type': 'limit', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'buy', 'price': 0.0685, 'stopPrice': None, 'triggerPrice': None, 'amount'

{'info': {'is_liquidation': False,
  'risk_reducing': False,
  'order_type': 'limit',
  'creation_timestamp': '1718797294825',
  'order_state': 'open',
  'contracts': '1.0',
  'original_order_type': 'market',
  'average_price': '0.0',
  'reduce_only': False,
  'post_only': False,
  'last_update_timestamp': '1718797294825',
  'filled_amount': '0.0',
  'replaced': False,
  'web': False,
  'api': True,
  'mmp': False,
  'instrument_name': 'ETH-27SEP24-17000-C',
  'order_id': 'ETH-8960119361',
  'max_show': '1.0',
  'time_in_force': 'good_til_cancelled',
  'amount': '1.0',
  'direction': 'buy',
  'price': '0.0685',
  'label': '',
  'trades': []},
 'id': 'ETH-8960119361',
 'clientOrderId': None,
 'timestamp': 1718797294825,
 'datetime': '2024-06-19T11:41:34.825Z',
 'lastTradeTimestamp': None,
 'symbol': 'ETH/USD:ETH-240927-17000-C',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 0.0685,
 'stopPrice': None,
 'triggerPrice': None,
 'amount': 1.0,
 'cost

In [14]:
share_to_purchase,symbol,delta,contract_size

(0.0325, 'ETH-27SEP24-17000-C', '0.0325', '1.0')

In [15]:
symbol=symbol.split("-")[0]
symbol=symbol+"USDT"
symbol

'ETHUSDT'

In [16]:
def create_hedging_order(symbol, side,share_to_purchase):
    """
    Create a hedging order on Binance testnet.

    Parameters:
    api_key (str): Your Binance testnet API key.
    api_secret (str): Your Binance testnet API secret.
    symbol (str): The trading pair symbol (e.g., 'BTCUSDT').
    side (str): 'BUY' or 'SELL' order.
    share_to_purchase (float): Amount of the asset to purchase.

    Returns:
    dict: Response from Binance API with order details.
    """
    # Connect to Binance testnet
    api_key,api_secret=get_keys("binance_keys.json")
    client = Client(api_key, api_secret, testnet=True)
    time_offset=get_time_off_set("time.json")
    if time_offset:
        client.timestamp_offset = time_offset
    # Get current price of the symbol
    ticker = client.get_symbol_ticker(symbol=symbol)
    current_price = float(ticker['price'])
    print(f"Current price of {symbol}: {current_price}")
    print(f"Share to purchase: {share_to_purchase} ")
    # Create a market order
    try:
        order = client.create_order(
            symbol=symbol,
            side=side,
            type='MARKET',
            quantity=float (share_to_purchase)
        )
        print("Order placed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        order = None
    
    return order



In [17]:
def get_min_contract_size(symbol):
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    data = response.json()

    for s in data['symbols']:
        if s['symbol'] == symbol:
            for filter in s['filters']:
                if filter['filterType'] == 'LOT_SIZE':
                    min_qty = filter['minQty']
                    return float(min_qty)
    return None

In [18]:
min_contract_size = get_min_contract_size(symbol)

if min_contract_size:
    print(f"The minimum contract size for {symbol} is {min_contract_size}")
else:
    print(f"Symbol {symbol} not found")

The minimum contract size for ETHUSDT is 0.0001


In [19]:
def calculate_and_place_order(symbol,old_delta):
    old_delta=float(old_delta)
    new_delta=float(information_for_options(client)[-2])
    contract_size=float(information_for_options(client)[-1])
    share_to_purchase = (new_delta - old_delta) * contract_size
    if share_to_purchase != 0:  # Only place an order if there's a change in delta
        if share_to_purchase <get_min_contract_size(symbol)*10:
            print(f"Share to purchase too small: {share_to_purchase}")
        else:
            if new_delta > old_delta:
                side = "BUY"
            else:
                side = "SELL"
            create_hedging_order(symbol,side, abs(share_to_purchase))
    else:
        print(f"{datetime.now()}: No change in delta. No order placed.")


In [20]:
create_hedging_order(symbol,side="SELL",share_to_purchase=get_min_contract_size(symbol)*100)

Current price of ETHUSDT: 3521.68
Share to purchase: 0.01 
Order placed successfully.


{'symbol': 'ETHUSDT',
 'orderId': 5512370,
 'orderListId': -1,
 'clientOrderId': 'Urvjg2EFWQqZOau5Ni55q3',
 'transactTime': 1718797316710,
 'price': '0.00000000',
 'origQty': '0.01000000',
 'executedQty': '0.01000000',
 'cummulativeQuoteQty': '35.22350000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1718797316710,
 'fills': [{'price': '3522.35000000',
   'qty': '0.01000000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 678837}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [21]:
def get_price_filters(symbol):
    """Retrieve price filter info for a given symbol."""
    base_url = 'https://testnet.binance.vision'
    endpoint = '/api/v3/exchangeInfo'
    response = requests.get(base_url + endpoint)
    data = response.json()

    for symbol_info in data['symbols']:
        if symbol_info['symbol'] == symbol:
            for filter in symbol_info['filters']:
                if filter['filterType'] == 'PRICE_FILTER':
                    return {
                        'minPrice': float(filter['minPrice']),
                        'maxPrice': float(filter['maxPrice']),
                        'tickSize': float(filter['tickSize'])
                    }
    raise Exception(f'Price filters not found for symbol: {symbol}')

In [22]:
get_price_filters(symbol)

{'minPrice': 0.01, 'maxPrice': 1000000.0, 'tickSize': 0.01}

In [23]:
def create_limit_order(symbol, side,share_to_purchase,limit_delta):
    """
    Create a hedging order on Binance testnet.

    Parameters:
    api_key (str): Your Binance testnet API key.
    api_secret (str): Your Binance testnet API secret.
    symbol (str): The trading pair symbol (e.g., 'BTCUSDT').
    side (str): 'BUY' or 'SELL' order.
    share_to_purchase (float): Amount of the asset to purchase.

    Returns:
    dict: Response from Binance API with order details.
    """
    # Connect to Binance testnet
    api_key,api_secret=get_keys("binance_keys.json")
    timestamp = int(time.time() * 1000)
    client = Client(api_key, api_secret, testnet=True)
    # Get current price of the symbol
    ticker = client.get_symbol_ticker(symbol=symbol)
    current_price = float(ticker['price'])
    print(f"Current price of {symbol}: {current_price}")
    print(f"Share to purchase: {share_to_purchase} ")
    print(f"Limited pirce : {round(current_price-current_price*limit_delta/100)}")
    # Create a market order
    try:
        order = client.create_order(
            symbol=symbol,
            side=side,
            type='LIMIT',
            price=round(current_price-current_price*limit_delta/100),
            timeInForce="GTC",
            timestamp=timestamp,
            quantity=float (share_to_purchase)
        )
        print("Order placed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        order = None
    
    return order

In [24]:


# Function to create a signature for the request
def create_signature(query_string, secret_key):
    return hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

# Function to get all open orders from Binance Spot Testnet
def get_all_open_orders():
    time_offset=get_time_off_set("time.json")
    if time_offset:
        client.timestamp_offset = time_offset
    api_key,api_secret=get_keys("binance_keys.json")
    base_url = 'https://testnet.binance.vision'
    endpoint = '/api/v3/openOrders'
    timestamp = int(time.time() * 1000)
    query_string = f'timestamp={timestamp+time_offset}'

    # Creating a signature
    signature = create_signature(query_string, api_secret)
    query_string += f'&signature={signature}'

    # Header with API key
    headers = {
        'X-MBX-APIKEY': api_key
    }

    # Sending the GET request
    url = f'{base_url}{endpoint}?{query_string}'
    response = requests.get(url, headers=headers)

    # Handling the response
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Error: {response.status_code}, Message: {response.text}')
def get_all_past_orders(symbol, start_time=None, end_time=None, limit=500):
    """Retrieve all past orders for a given symbol on Binance Spot Testnet."""
    time_offset=get_time_off_set("time.json")
    if time_offset:
        client.timestamp_offset = time_offset
    base_url = 'https://testnet.binance.vision'
    endpoint = '/api/v3/allOrders'
    timestamp = int(time.time() * 1000)
    api_key,api_secret=get_keys("binance_keys.json")  
    # Create query string
    query_string = f'symbol={symbol}&timestamp={timestamp+time_offset}&limit={limit}'
    if start_time:
        query_string += f'&startTime={start_time}'
    if end_time:
        query_string += f'&endTime={end_time}'

    # Create signature
    signature = create_signature(query_string, api_secret)
    query_string += f'&signature={signature}'

    # Headers
    headers = {'X-MBX-APIKEY': api_key}

    # Send GET request
    url = f'{base_url}{endpoint}?{query_string}'
    response = requests.get(url, headers=headers)

    # Handle the response
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Error: {response.status_code}, Message: {response.text}')


In [25]:
def get_account_balances(symbol=None):
    api_key,api_secret=get_keys("binance_keys.json")
    client = Client(api_key, api_secret, testnet=True)
    time_offset=get_time_off_set("time.json")
    if time_offset:
        client.timestamp_offset = time_offset
    try:
        # Fetch account information
        account_info = client.get_account()

        # Extract balances
        balances = account_info['balances']
        balance_info = []

        # Iterate through each balance and append non-zero balances to the list
        if symbol==None:
            for balance in balances:
                asset = balance['asset']
                free_balance = float(balance['free'])
                locked_balance = float(balance['locked'])
                total_balance = free_balance + locked_balance

                if total_balance > 0:
                    balance_info.append({
                        'Asset': asset,
                        'Free': free_balance,
                        'Locked': locked_balance,
                        'Total': total_balance
                    })
        else: 
            for balance in balances:
                if balance['asset'] in symbol:
                    asset = balance['asset']
                    free_balance = float(balance['free'])
                    locked_balance = float(balance['locked'])
                    total_balance = free_balance + locked_balance

                    if total_balance > 0:
                        balance_info.append({
                            'Asset': asset,
                            'Free': free_balance,
                            'Locked': locked_balance,
                            'Total': total_balance
                        })

        return balance_info

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [81]:
past_orders=get_all_past_orders(symbol)
past_orders_df=pandas.DataFrame(past_orders)
past_orders_df.to_csv("past_order.csv")

In [30]:
current_timestamp = time.time()

In [48]:
import pandas as pd
from datetime import datetime

def timestamp_and_save_data(balance_sheets):
    """
    Appends balance sheet data with a timestamp to the CSV file.
    """
    # Get current timestamp
    current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    



In [111]:
json_file_path = 'nested_dataframes.json'


In [112]:
with open(json_file_path, 'r') as json_file:
    nested_bal_dict = json.load(json_file)

In [107]:
store_balances_dict(nested_bal_dict)

{'2024-06-19 12:35:49': [{'Asset': 'ETH',
   'Free': 2.94,
   'Locked': 0.0,
   'Total': 2.94},
  {'Asset': 'BTC', 'Free': 1.02604, 'Locked': 0.0, 'Total': 1.02604},
  {'Asset': 'LTC', 'Free': 5.0, 'Locked': 0.0, 'Total': 5.0},
  {'Asset': 'BNB', 'Free': 1.0, 'Locked': 0.0, 'Total': 1.0},
  {'Asset': 'USDT',
   'Free': 1157.0442307,
   'Locked': 0.0,
   'Total': 1157.0442307},
  {'Asset': 'TRX', 'Free': 4386.0, 'Locked': 0.0, 'Total': 4386.0},
  {'Asset': 'XRP', 'Free': 948.0, 'Locked': 0.0, 'Total': 948.0},
  {'Asset': 'NEO', 'Free': 33.0, 'Locked': 0.0, 'Total': 33.0},
  {'Asset': 'QTUM', 'Free': 141.0, 'Locked': 0.0, 'Total': 141.0},
  {'Asset': 'EOS', 'Free': 617.0, 'Locked': 0.0, 'Total': 617.0},
  {'Asset': 'SNT', 'Free': 14258.0, 'Locked': 0.0, 'Total': 14258.0},
  {'Asset': 'BNT', 'Free': 606.0, 'Locked': 0.0, 'Total': 606.0},
  {'Asset': 'GAS', 'Free': 98.0, 'Locked': 0.0, 'Total': 98.0},
  {'Asset': 'LRC', 'Free': 1850.0, 'Locked': 0.0, 'Total': 1850.0},
  {'Asset': 'ZRX', 'F

In [108]:
len(nested_bal_dict)

3

In [110]:
json_file_path = 'nested_dataframes.json'
with open(json_file_path, 'w') as json_file:
    json.dump(nested_bal_dict, json_file, indent=4)

In [115]:
def update_time_index_balance_sheet(json_file_path):
    with open(json_file_path, 'r') as json_file:
        nested_bal_dict = json.load(json_file)
    balances=get_account_balances()
    current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    nested_bal_dict[current_timestamp]=balances
    with open(json_file_path, 'w') as json_file:
        json.dump(nested_bal_dict, json_file, indent=4)


In [23]:
import schedule
import time

# Assuming these are updated dynamically from a data source

with open("old_delta.json", 'r') as file:

    data = json.load(file)
old_delta=data
def job():
    try: 
        global old_delta
        print("Performing job")
        calculate_and_place_order(symbol,old_delta)
        old_delta=information_for_options(client)[-2]
        print(f"Updated old_delta to: {old_delta}")       
        with open("old_delta.json", "w") as json_file:
            json.dump(old_delta, json_file)
    except Exception as e:
        print(f"Exception: {e}")

    
# Schedule the job every hour
schedule.every(10).seconds.do(job)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)

Performing job
2024-06-17 13:51:40.897831: No change in delta. No order placed.
Updated old_delta to: 0.0009
Performing job
2024-06-17 13:51:55.468693: No change in delta. No order placed.
Updated old_delta to: 0.00089


KeyboardInterrupt: 

In [97]:
def get_account_balances(symbol=None):
    api_key,api_secret=get_keys("binance_keys.json")
    client = Client(api_key, api_secret, testnet=True)
    time_offset=get_time_off_set("time.json")
    if time_offset:
        client.timestamp_offset = time_offset
    try:
        # Fetch account information
        account_info = client.get_account()

        # Extract balances
        balances = account_info['balances']
        balance_info = []

        # Iterate through each balance and append non-zero balances to the list
        if symbol==None:
            for balance in balances:
                asset = balance['asset']
                free_balance = float(balance['free'])
                locked_balance = float(balance['locked'])
                total_balance = free_balance + locked_balance

                if total_balance > 0:
                    balance_info.append({
                        'Asset': asset,
                        'Free': free_balance,
                        'Locked': locked_balance,
                        'Total': total_balance
                    })
        else: 
            for balance in balances:
                if balance['asset'] in symbol:
                    asset = balance['asset']
                    free_balance = float(balance['free'])
                    locked_balance = float(balance['locked'])
                    total_balance = free_balance + locked_balance

                    if total_balance > 0:
                        balance_info.append({
                            'Asset': asset,
                            'Free': free_balance,
                            'Locked': locked_balance,
                            'Total': total_balance
                        })

        return balance_info

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [98]:
def display_balances(balances):
    time_offset=get_time_off_set("time.json")
    if time_offset:
        client.timestamp_offset = time_offset
    if not balances:
        print("No balances found or an error occurred.")
        return

    print(f"{'Asset':<10} {'Free Balance':<15} {'Locked Balance':<15} {'Total Balance':<15}")
    print("="*55)
    for balance in balances:
        print(f"{balance['Asset']:<10} {balance['Free']:<15} {balance['Locked']:<15} {balance['Total']:<15}")

In [104]:
list=["ETH","USDT"]
balances = get_account_balances(list)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2319792559.py, line 1)

In [105]:
display_balances(balances)

Asset      Free Balance    Locked Balance  Total Balance  
ETH        2.95            0.0             2.95           
USDT       1121.8207307    0.0             1121.8207307   
